# Keras Tuner-Decide Number of Hidden Layers And Neurons In Neural Network 

In [7]:
import tensorflow as tf
import pandas as pd
from kerastuner.tuners import RandomSearch

C:\Users\rocky\AppData\Local\Temp/ipykernel_6964/2084613809.py:3: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


In [2]:
df = pd.read_csv('Real_Combine.csv')
# Air-quality index prediction
df.head()

,T,TM,Tm,SLP,H,VV,V,VM,PM 2.5
0,7.4,9.8,4.8,1017.6,93.0,0.5,4.3,9.4,219.720833
1,7.8,12.7,4.4,1018.5,87.0,0.6,4.4,11.1,182.187500
2,6.7,13.4,2.4,1019.4,82.0,0.6,4.8,11.1,154.037500
3,8.6,15.5,3.3,1018.7,72.0,0.8,8.1,20.6,223.208333
4,12.4,20.9,4.4,1017.3,61.0,1.3,8.7,22.2,200.645833


In [3]:
X = df.iloc[:,:-1] ## independent features
y = df.iloc[:,-1] ## dependent features

In [23]:
## Custom function for deciding no of neurons and layers

# Creating a model
def build_model(hp):
    model = tf.keras.Sequential()
    for i in range(hp.Int('num_layers', 2, 20)): # This particular neural network can have range of 2 to 20
        model.add(tf.keras.layers.Dense(units=hp.Int('units_' + str(i),
                                                    min_value = 32, # NO of neurons ranging from 32 to 512
                                                    max_value = 512,
                                                    step = 32),
                                       activation='relu'))
        model.add(tf.keras.layers.Dense(1, activation='linear'))
        
        ## Compile a model     
        model.compile(optimizer = tf.keras.optimizers.Adam(hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), # Learning Rates
                      loss= 'mean_absolute_error',
                      metrics=['mean_absolute_error'])
    return model

In [10]:
tuner = RandomSearch(
    build_model,
    objective='val_mean_absolute_error',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='Air Quality Index')

In [11]:
tuner.search_space_summary()

Search space summary
Default search space size: 4
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': None}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': None}


In [12]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=0)

In [13]:
## Fit the model
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

Trial 5 Complete [00h 00m 08s]
val_mean_absolute_error: nan

Best val_mean_absolute_error So Far: nan
Total elapsed time: 00h 00m 57s
INFO:tensorflow:Oracle triggered exit


In [28]:
tuner.results_summary()

Results summary
Results in .\untitled_project
Showing 10 best trials
Trial summary
Hyperparameters:
num_layers: 6
units_0: 64
learning_rate: 0.0001
units_1: 416
units_2: 96
units_3: 224
units_4: 416
units_5: 256
units_6: 320
units_7: 288
units_8: 320
units_9: 416
units_10: 224
units_11: 352
units_12: 480
units_13: 416
Score: nan
Trial summary
Hyperparameters:
num_layers: 5
units_0: 320
learning_rate: 0.0001
units_1: 384
units_2: 384
units_3: 448
units_4: 416
units_5: 416
units_6: 288
units_7: 96
units_8: 288
units_9: 128
units_10: 224
units_11: 352
units_12: 192
units_13: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 14
units_0: 128
learning_rate: 0.001
units_1: 224
units_2: 32
units_3: 32
units_4: 32
units_5: 32
units_6: 32
units_7: 32
units_8: 32
units_9: 32
units_10: 32
units_11: 32
units_12: 32
units_13: 32
Score: nan
Trial summary
Hyperparameters:
num_layers: 10
units_0: 480
learning_rate: 0.01
units_1: 480
units_2: 384
units_3: 192
units_4: 512
units_5: 256
units_6: 35